# Can we use Claude to extract all the fields from a form

Based on the idea from here: 
https://github.com/co-cddo/form-extractor-prototype_testing/tree/main

Yes we can.

In [1]:
%load_ext autoreload
%autoreload 2

import json
import logging

import boto3
import nest_asyncio
import pandas as pd
from anthropic import AsyncAnthropicBedrock
from dotenv import load_dotenv

from form_processor import (
    compute_total,
    get_list_of_file_paths,
    load_batch_results,
    run_form_processing,
    test_client,
)

logging.basicConfig(
    filename="form_processing.log",
    level=logging.INFO,  # Adjust the level as needed (e.g., INFO, DEBUG, WARNING, ERROR, CRITICAL)
    format="%(asctime)s - %(levelname)s - %(message)s",
    filemode="a",  # 'a' for append mode, 'w' for overwrite mode
)
logging.info("Setup log")

In [2]:
logging.info("Intitiating log")

load_dotenv()

nest_asyncio.apply()

boto3.setup_default_session()
client = AsyncAnthropicBedrock()
await test_client(client)


True

In [3]:

with open("extract-form-questions.json", "r") as file:
    json_string = file.read()

extraction_tool = json.loads(json_string)

folder_path = "sample_forms"
pdf_files = get_list_of_file_paths(folder_path)

folder_path = "sample_forms"
results = run_form_processing(
    pdf_files=pdf_files,
    client=client,
    extraction_tool=extraction_tool,
    batch_size=10,
    max_concurrent=10,
    results_file="test_folder.pickle",
)



Processing forms. Check 'form_processing.log' for detailed logs.


Processing Forms: 100%|██████████| 4/4 [01:05<00:00, 16.39s/form]

Processed 4 forms


In [4]:
results = load_batch_results("test_folder.pickle")

In [5]:
results_df = pd.DataFrame.from_dict(results, orient="index")

NameError: name 'pd' is not defined

In [ ]:
compute_total(results_df)


In [ ]:
# TODO modify the method so it can take a list of files